<h2 style="color:#1f77b4;">Lección 3: Obtención de datos desde archivos</h2>

<h3>Parte 1</h3>

<h4>Leyendo un archivo CSV</h4>

In [ ]:
import pandas as pd

# Leer un archivo CSV básico
df = pd.read_csv('Titanic-Dataset.csv')

# Mostrar las primeras filas
df.head()


<h5>Opciones para leer archivos CSV</h5>

<p>1. Leer un archivo CSV con punto y coma como delimitador</p>

In [ ]:
df = pd.read_csv("datos_punto_y_coma.csv", sep=";")
print(df.head())

<p>2. Leer un archivo CSV con tabulación como delimitador</p>

In [ ]:
df = pd.read_csv("datos_tab.csv", sep="\t")
print(df.head())

<h4>Escribiendo un archivo CSV</h4>
Para guardar un DataFrame en un archivo CSV, se utiliza el método to_csv().

In [ ]:
df = pd.DataFrame({
    "id": [1, 2, 3, 4],
    "nombre": ["Ana", "Juan", "María", "Pedro"],
    "edad": [25, 30, 28, 35],
    "ventas": [1200, 1500, 1100, 1800]
})

print("DataFrame original:")
print(df)

# 2) Guardar un DataFrame en un archivo CSV (incluye el índice por defecto)
df.to_csv("nuevo_datos.csv")
print("\nSe creó: nuevo_datos.csv (con índice)")

# 3) Guardar sin incluir el índice
df.to_csv("nuevo_datos_sin_indice.csv", index=False)
print("Se creó: nuevo_datos_sin_indice.csv (sin índice)")


<h3>Archivos Excel</h3>

In [ ]:
import pandas as pd

excel_path="Valor del Dolar.xlsx"

# Leer un archivo Excel
df = pd.read_excel(excel_path)

# Mostrar las primeras filas
print(df.head())


In [ ]:
# si da error Instalar -> pip install openpyxl


In [ ]:
# 1) Ver hojas disponibles
xls = pd.ExcelFile(excel_path)
print("Hojas disponibles:", xls.sheet_names)

In [ ]:
# 2) Leer una hoja por nombre (ajusta el nombre real)
df = pd.read_excel(excel_path, sheet_name=xls.sheet_names[0])
display(df.head())

In [ ]:
# 3) Leer una hoja por índice
df0 = pd.read_excel(excel_path, sheet_name=0)
df0

In [ ]:
# 4) Leer múltiples hojas (diccionario de DataFrames)
dfs = pd.read_excel(excel_path, sheet_name=xls.sheet_names[:2])
print("Hojas cargadas:", list(dfs.keys()))

In [ ]:
dfs

<h2>Leer tablas web con pandas</h2>

In [ ]:
#%pip install requests

In [ ]:
#%pip install lxml


In [ ]:
import pandas as pd
import requests
from io import StringIO

url = "https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_por_PIB_(PPA)_per_c%C3%A1pita"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept-Language": "es-ES,es;q=0.9"
}

html = requests.get(url, headers=headers, timeout=30).text
df_list = pd.read_html(StringIO(html))

# Ver cuántas tablas y una “vista rápida” de cada una
print("Cantidad de tablas:", len(df_list))
for i, t in enumerate(df_list):
    print(i, t.shape, "->", list(t.columns)[:6])

In [ ]:
df = df_list[2].copy()
df.head()

In [ ]:
# 5) Renombrar columnas (explícito)
df = df.rename(columns={
    "Pos.": "posicion",
    "País": "pais",
    "PIB per cápita (PPA) Dólares internacionales": "pib_ppa_per_capita",
    "Δ": "delta"
})
df.head()

In [ ]:
# Limpieza
# - quitar referencias tipo [1], [nota 2], etc.
df = df.replace(r"\[.*?\]", "", regex=True)
df.head()

In [ ]:
# - limpiar espacios raros
df["pais"] = df["pais"].astype(str).str.strip()
df.head(100)

In [ ]:
# Convertir pib a número (quita separadores y cosas no numéricas)
df["pib_ppa_per_capita"] = (
    df["pib_ppa_per_capita"]
    .astype(str)                                # fuerza a que todos los valores de la columna pais sean texto
    .str.replace(r"[^\d,.\-]", "", regex=True)  # deja solo dígitos y signos
    .str.replace(".", "", regex=False)          # por si viene con separador de miles "."
    .str.replace(",", "", regex=False)          # por si viene con separador de miles ","
)
df.head(100)

In [ ]:
# Convertir pib a número 
df["pib_ppa_per_capita"] = pd.to_numeric(df["pib_ppa_per_capita"], errors="coerce")
df.head(100)

In [ ]:
# - convertir posición a número
df["posicion"] = pd.to_numeric(df["posicion"], errors="coerce")
df.head(100)

In [ ]:
# - eliminar filas con faltantes relevantes
df = df.dropna(subset=["posicion", "pais", "pib_ppa_per_capita"])
df.head(100)

1) sort_values("posicion"): Ordena las filas según la columna posicion.
2) reset_index(drop=True): reset_index() crea un índice nuevo 0,1,2,3… y drop=True significa: No guardes el índice anterior como una columna extra

In [ ]:
# Ordenar por posición
df = df.sort_values("posicion").reset_index(drop=True)
df.head(100)

In [ ]:
# 7) Guardar a CSV
df.to_csv("pib_ppa_per_capita.csv", index=False, encoding="utf-8")

<h3>Parte 2</h3>

Datos desde https://datosabiertos.fonasa.cl/

Archivo_desde = "https://public.tableau.com/views/PropuestaTableroGRD/PropuestaTableroGRD?%3AshowVizHome=no#1"


<h3>Lectura por fragmentos (chunks)</h3>

<p>chunks es una forma de leer el archivo por partes, en vez de cargarlo completo en memoria.</p>
<p>Pandas no te entrega un DataFrame único. 
Te entrega un iterador que va produciendo DataFrames de 1000 filas cada vez.
</p>

En la práctica significa:
* lees 1000 filas → las procesas → sigues con las siguientes 1000
* así puedes trabajar con archivos grandes sin “reventar” la RAM
* y puedes ir guardando resultados a medida que avanzas

In [ ]:
# Leer un archivo CSV en fragmentos de 1000 filas
chunks = pd.read_csv(
    "GRD_PUBLICO_2019.txt",
    sep="|",                 # ajusta si corresponde
    encoding="latin1",
    chunksize=1000,
    engine="python",         # más tolerante que el C engine
    on_bad_lines="skip"     # salta filas problemáticas
)

In [ ]:
chunk1 = next(chunks)  # primeras 1000
chunk1.shape

In [ ]:
chunk1.head()

In [ ]:
chunk2 = next(chunks)  # siguientes 1000
chunk2.shape

In [ ]:
chunk1.to_csv("chunk1.csv", index=False, encoding="utf-8")

<h3>Selección de columnas específicas</h3>

In [ ]:
# Leer solo las columnas necesarias
columnas = ['CIP_ENCRIPTADO','SEXO','FECHA_NACIMIENTO']
df = pd.read_csv('chunk1.csv', usecols=columnas)
df.head()


<h3>Especificación de tipos de datos</h3>
Definir los tipos de datos de antemano puede mejorar significativamente el rendimiento y reducir el uso de memoria:

Especificar tipos de datos precisos puede reducir el uso de memoria hasta en un 50% en algunos casos

In [ ]:
# Definir tipos de datos para cada columna
tipos = {
    'COD_HOSPITAL': 'int32',
    'CIP_ENCRIPTADO': 'int32'
}

# Leer el archivo con los tipos especificados
df = pd.read_csv('chunk1.csv', dtype=tipos, parse_dates=['FECHA_NACIMIENTO'])
df.head()


<h3>Filtrado durante la lectura</h3>
Es posible aplicar filtros durante la lectura para cargar solo las filas que cumplen ciertas condiciones:

In [ ]:
# Leer solo las filas que cumplen la condición
df = pd.read_csv('chunk1.csv', 
                chunksize=1000, 
                iterator=True)
df_filtrado = pd.concat([chunk[chunk['SEXO'] == "MUJER"] for chunk in df])


In [ ]:
df_filtrado.shape

In [ ]:
df_filtrado["SEXO"].unique()

<h3>Ejemplo de integración completa</h3>

In [ ]:
import pandas as pd

data = [
    {"Gasto": 120.5, "Frecuencia": 2, "Categoría": "Básico"},
    {"Gasto": 85.0, "Frecuencia": 1, "Categoría": "Básico"},
    {"Gasto": 300.0, "Frecuencia": 4, "Categoría": "Premium"},
    {"Gasto": 450.75, "Frecuencia": 6, "Categoría": "Premium"},
    {"Gasto": 210.2, "Frecuencia": 3, "Categoría": "Estándar"},
    {"Gasto": 190.0, "Frecuencia": 3, "Categoría": "Estándar"},
    {"Gasto": 560.0, "Frecuencia": 7, "Categoría": "Premium"},
    {"Gasto": 95.5, "Frecuencia": 2, "Categoría": "Básico"},
    {"Gasto": None, "Frecuencia": 5, "Categoría": "Estándar"},
    {"Gasto": 180.0, "Frecuencia": None, "Categoría": "Básico"},
    {"Gasto": 275.0, "Frecuencia": 4, "Categoría": "Estándar"},
    {"Gasto": 620.3, "Frecuencia": 8, "Categoría": "Premium"},
]

pd.DataFrame(data).to_csv("clientes.csv", index=False, encoding="utf-8")


In [ ]:
#%pip install seaborn

In [ ]:
#%pip install scikit-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

# 1. Obtener datos
df = pd.read_csv('clientes.csv')

# 2. Procesar datos
df = df.dropna()
df['Gasto'] = df['Gasto'].astype(float)
df['Frecuencia'] = df['Frecuencia'].astype(int)

# 3. Análisis exploratorio
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Gasto', y='Frecuencia', hue='Categoría')
plt.title('Relación entre Gasto y Frecuencia por Categoría')
plt.savefig('analisis_exploratorio.png')

# 4. Aplicar modelo
X = df[['Gasto', 'Frecuencia']]
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(X)

# 5. Guardar resultados
df.to_excel('clientes_segmentados.xlsx', index=False)


<h3>Manejo de datos faltantes</h3>

In [ ]:
df = pd.read_csv('clientes.csv')
df


In [ ]:
# Detectar valores faltantes
df.isnull().sum()


In [ ]:
# Eliminar filas con valores faltantes
df_limpio = df.dropna()
df_limpio.isnull().sum()


In [ ]:
df_limpio

In [ ]:
df_relleno = df.fillna(0)  # Con ceros
df_relleno.isnull().sum()

In [ ]:
df_relleno

In [ ]:
df_relleno = df.copy()

In [ ]:
df_relleno.describe()

In [ ]:

df_relleno = df.fillna(df.mean(numeric_only=True)) # Eso calcula el promedio solo de las columnas numéricas (Gasto, Frecuencia) y rellena sus NaN con ese promedio.


In [ ]:
df_relleno

<h3>Detección y manejo de valores atípicos</h3>

<p>Tenemos un listado de clientes con su gasto mensual.
La mayoría gasta montos razonables… pero hay uno o dos valores raros, muy altos, que pueden distorsionar el análisis.

Eso es exactamente lo que llamamos valores atípicos (outliers).</p>

In [ ]:
#Creamos un dataset de ejemplo
df = pd.DataFrame({
    "Cliente": ["A", "B", "C", "D", "E", "F", "G", "H"],
    "Gasto":   [120, 150, 130, 140, 160, 155, 145, 1200]
})

df

<h4>Detección de outliers con el método IQR</h4>

Entendiendo la idea del método IQR

* Q1 (25%): valores bajos normales
* Q3 (75%): valores altos normales
* IQR = rango “normal” del dato

Todo lo que quede muy lejos de ese rango se considera atípico.

Calcular Q1, Q3 e IQR

In [ ]:
Q1 = df["Gasto"].quantile(0.25)
Q3 = df["Gasto"].quantile(0.75)
IQR = Q3 - Q1

Q1, Q3, IQR

Definir límites

In [ ]:
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR


Identificar los outliers

In [ ]:
outliers = df[
    (df["Gasto"] < limite_inferior) |
    (df["Gasto"] > limite_superior)
]

outliers

¿Qué hacemos con los outliers?

In [ ]:
# 1 Eliminarlos
df_sin_outliers = df[
    (df["Gasto"] >= limite_inferior) &
    (df["Gasto"] <= limite_superior)
]

df_sin_outliers


In [ ]:
# 2 Reemplazarlos
df_reemplazado = df.copy()

# Convertir en el mismo df donde vas a reemplazar
df_reemplazado["Gasto"] = pd.to_numeric(df_reemplazado["Gasto"], errors="coerce").astype(float)

mediana = df_reemplazado["Gasto"].median()

mask = (df_reemplazado["Gasto"] < limite_inferior) | (df_reemplazado["Gasto"] > limite_superior)
df_reemplazado.loc[mask, "Gasto"] = float(mediana)

df_reemplazado



<h3>Formatos alternativos para grandes volúmenes de datos</h3>

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "id": [1, 2, 3, 4],
    "cliente": ["A", "B", "C", "D"],
    "gasto": [120.5, 85.0, 300.0, 450.75],
    "fecha": pd.to_datetime(["2026-01-01", "2026-01-02", "2026-01-03", "2026-01-04"])
})

df


<h4>Parquet (ideal para análisis, comprimido, rápido)</h4>

In [ ]:
#pip install pyarrow

In [ ]:
# Guardar el DataFrame en formato Parquet
df.to_parquet("datos.parquet", index=False) # guardar
df_parquet = pd.read_parquet("datos.parquet") # leer
df_parquet.head()


HDF5 (útil para guardar “datasets” grandes y estructurados)

In [ ]:
#pip install tables


In [ ]:
df.to_hdf("datos.h5", key="df", mode="w")      # guardar
df_hdf = pd.read_hdf("datos.h5", key="df")     # leer
df_hdf.head()


Feather (muy rápido, intercambio Python ↔ R, también usa Arrow)

In [ ]:
# pip install pyarrow

In [ ]:
df.to_feather("datos.feather")            # guardar
df_feather = pd.read_feather("datos.feather")  # leer
df_feather.head()

<h3>Conexión con bases de datos</h3>
SQLite viene incluido por defecto con Python.

SQLite en 1 minuto

¿Qué es?
SQLite es una base de datos relacional que vive en un archivo .db.
No necesita servidor (a diferencia de PostgreSQL/MySQL). Ideal para demos, proyectos pequeños y prototipos.

In [ ]:
import sqlite3
print(sqlite3.sqlite_version)


In [ ]:
import sqlite3
import pandas as pd

# Conectar (o crear) base
conn = sqlite3.connect("prueba.db")

# Crear DataFrame simple
df = pd.DataFrame({
    "id": [1, 2, 3],
    "nombre": ["Ana", "Juan", "María"]
})

# Guardar en SQLite
df.to_sql("personas", conn, if_exists="replace", index=False)

# Leer desde SQLite
df_leido = pd.read_sql("SELECT * FROM personas", conn)
print(df_leido)

conn.close()


<h4>SQLite desde Python</h4>

Insertar un registro

In [ ]:
conn = sqlite3.connect("prueba.db")
conn.execute("INSERT INTO personas (id, nombre) VALUES (?, ?)", (4, "Sofía"))
conn.commit()
conn.close()


In [ ]:
# Leer desde SQLite
conn = sqlite3.connect("prueba.db")
df_leido = pd.read_sql("SELECT * FROM personas", conn)
print(df_leido)

Actualizar

In [ ]:
conn = sqlite3.connect("prueba.db")
conn.execute("UPDATE personas SET nombre = ? WHERE id = ?", ("Juan Pablo", 2))
conn.commit()
conn.close()


In [ ]:
# Leer desde SQLite
conn = sqlite3.connect("prueba.db")
df_leido = pd.read_sql("SELECT * FROM personas", conn)
print(df_leido)

Eliminar

In [ ]:
conn = sqlite3.connect("prueba.db")
conn.execute("DELETE FROM personas WHERE id = ?", (1,))
conn.commit()
conn.close()


In [ ]:
conn = sqlite3.connect("prueba.db")
df = pd.read_sql("SELECT * FROM personas", conn)
conn.close()
df


<h3>Obtención de datos desde APIs</h3>
Muchos servicios web ofrecen APIs que permiten acceder a datos de forma programática. Pandas puede trabajar con estos datos una vez obtenidos:

In [ ]:
import requests
import pandas as pd

# Realizar solicitud a la API
url = 'https://api.exchangerate-api.com/v4/latest/USD'
respuesta = requests.get(url)
datos = respuesta.json()

# Convertir a DataFrame
df = pd.DataFrame(datos)
df


In [ ]:
# Procesar y guardar
df.to_csv('datos_api.csv', index=False)

<h3>Ejemplo de obtención desde Yahoo Finance</h3>

In [ ]:
#pip install yfinance

In [ ]:
import pandas as pd
import yfinance as yf

# Obtener datos históricos de una acción
ticker = 'AAPL'
periodo = '1y' # 1 año
df = yf.download(ticker, period=periodo)

# Mostrar los primeros registros
print(df.head())

# Guardar en CSV
df.to_csv(f'{ticker}_historico.csv')

# Calcular rendimientos diarios
df['Rendimiento'] = df['Close'].pct_change() * 100

# Visualizar rendimientos
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
df['Rendimiento'].plot()
plt.title(f'Rendimiento diario de {ticker}')
plt.savefig(f'{ticker}_rendimiento.png')
